In [1]:
from sampo_api import *
from prompts import * 
from models import *
#
import os
from sampo.schemas.graph import WorkGraph
from sampo.scheduler.genetic.operators import TimeFitness
import json

from sampo.scheduler.genetic.operators import is_chromosome_correct

def dict2string(dict):
    return json.dumps({str(k): list(v) for k, v in dict.items()})


wg = WorkGraph.loadf(f'wgs/small_synth', f'wg_22')
contractors = contractor(N=9)
children = take_children(wg)
activity_graph = dict2string(children)
init_choromosome = first_population(wg, contractors)
toolbox = create_mvp_toolbox(wg, contractors)


Can not find native module; switching to default


In [2]:
_, _, R = chromosome_to_text(init_choromosome['heft_end'][0])

In [3]:
print(R)

<R>6 6 8 2 3 2
13 12 16 4 6 4
19 19 24 6 9 6
23 22 31 8 11 8
31 29 35 10 14 10
35 35 45 12 17 12
42 41 56 14 21 13
47 49 69 17 23 16
55 58 68 19 26 19</R>


In [4]:
# min_v = 100
# for file in os.listdir('wgs/small_synth'):
#     wg = WorkGraph.loadf(f'wgs/small_synth', file[:-5])
#     children = take_children(wg)
#     activity_graph = dict2string(children)
#     min_v = min(min_v, len(children))
#     #print(len)
#     if min_v == len(children):
#         print(file, min_v)


In [5]:
# from sampo.scheduler.genetic import GeneticScheduler

# scheduler = GeneticScheduler(mutate_order=0.2,
#                              mutate_resources=0.1)

# schedule = scheduler.schedule(wg, contractors)[0]

# oracle_solve = schedule.execution_time 

In [6]:
# oracle_solve

In [7]:
# print('Optim solve', oracle_solve)

In [8]:
import re
with open('prompts/1.txt', 'r', encoding='utf-8') as file:
    content = file.read()
init_prompt = re.findall('<start>(.+?)</start>', content, re.DOTALL)[0]
generate_prompt = re.findall('<push>(.+?)</push>', content, re.DOTALL)[0]
system_prompt = init_prompt.format(R, activity_graph) 

In [9]:
# print(generate_prompt)
# print(system_prompt)

In [ ]:
# TODO - для бест решения составить какое то хранение, потому что иначе он затирается

def llm_genetic_algorithm(
    initial_population, 
    toolbox, 
    llm_model,
    max_generations=2,
    elite_size=7,
    max_plateau=3,
    f = TimeFitness()):
    """
    Упрощенный генетический алгоритм с селекцией и отслеживанием плато
    """

    # Оцениваем начальную популяцию
    population = []
    for chrom in initial_population:
        fitness = f.evaluate(chrom, evaluator=toolbox.evaluate_chromosome)
        population.append((chrom, fitness, False))
        
    B, land, H = initial_population[0][2], initial_population[0][3], initial_population[0][4] 
    # Сортируем по fitness (чем выше, тем лучше)
    population.sort(key=lambda x: x[1], reverse=True)
    
    # Инициализация
    generation = 0
    plateau = 0
    best_fitness = population[0][1]
    bad_cnt = 0
    # Главный цикл
    while generation < max_generations and plateau < max_plateau:
        
        # Элитарный отбор
        elites = population[:elite_size]
        
        # Генерация потомков
        offsprings = []
        fitness = None
        # Случайные пары для кроссинговера
        idxs = np.arange(len(population))
        np.random.shuffle(idxs)
        print(f'{generation} Поколение генерируется')
        for i in range(0, len(idxs)-1, 2):
            # Берем двух случайных родителей
            idx1, idx2 = idxs[i], idxs[i+1]
            chrom1, _, _ = population[idx1]
            chrom2, _, _ = population[idx2]
            
            # Кроссинговер
      
            str1 = chromosome_to_text(chrom1)
            str2 = chromosome_to_text(chrom2)
            offspring_str = llm_model.generate(str1, str2) 
            print(offspring_str)
            TA, RA = parse_chromosome(offspring_str)
            offspring_chrom  = ChromosomeType((np.array(TA), np.array(RA), np.array(B), land, H))
            try:
                if check_chromosome(wg, contractors, offspring_chrom):
                    fitness = f.evaluate(offspring_chrom, evaluator=toolbox.evaluate_chromosome)
                    print(f'В {generation} поколении успешно сгенерирован допустимый кандидат')
                    offsprings.append((offspring_chrom, fitness, True))
                else:
                    print(f'В {generation} поколении ошибка генерации')
            except:
                bad_cnt +=1
                continue
        
        # Если потомков недостаточно, добавляем копии случайных родителей
        if len(offsprings) < len(population) - elite_size:
            needed = (len(population) - elite_size) - len(offsprings)
            for _ in range(needed):
                idx = np.random.randint(0, len(population))
                chrom, fit, is_gen = population[idx]
                offsprings.append((chrom, fit, is_gen))
        
        # Объединяем и сортируем
        combined = elites + offsprings
        combined.sort(key=lambda x: x[1], reverse=True)
        new_population = combined[:len(population)]
        
        # Проверяем улучшение
        new_best = new_population[0][1]
        if new_best > best_fitness:
            print(new_best)
            best_fitness = new_best
            plateau = 0
        else:
            plateau += 1
            
            # При длительном плато увеличиваем разнообразие
            if plateau > max_plateau // 2:
                break
                # # Заменяем худшую особь
                # if len(new_population) > elite_size:
                #     # Генерируем новую случайную хромосому или мутируем
                #     idx = np.random.randint(elite_size, len(population))
                #     # new_chrom = generate_random_chromosome() или mutate()
                #     # new_fitness = toolbox.evaluate_chromosome(new_chrom)
                #     # new_population[-1] = (new_chrom, new_fitness)
                #     pass
        
        population = new_population
        generation += 1
    
    # Финальная сортировка
    population.sort(key=lambda x: x[1], reverse=True)
    return population, bad_cnt

In [ ]:
population = []

for chrom_d in list(init_choromosome.values()):
    chrom, _, _ = chrom_d
    population.append(chrom)
from dotenv import load_dotenv
load_dotenv()

N_gen = 1

api_key = os.getenv("DEEPSEEK_API_KEY")
    
llm_model = DeepSeekSession(
        api_key=api_key,
        generate_prompt=generate_prompt,
        system_prompt=system_prompt,
        max_history=100
        )

population_g, cnt = llm_genetic_algorithm(population, toolbox, llm_model)

0 Поколение генерируется
Plan 0
<TA>0,1,2,8,3,16,14,18,15,11,20,22,7,12,21,5,24,25,26,27,28,19,23,6,9,13,10,4,17,29,30</TA>
<RA>0 0 0 0 0 0 0
18 0 23 0 8 0 2
30 0 34 0 13 0 4
18 0 23 0 5 5 2
3 3 3 2 2 0 3
10 10 10 7 7 0 4
7 7 7 4 4 0 6
5 8 11 3 1 0 4
34 0 44 0 16 0 5
7 7 7 4 4 0 6
3 3 3 2 2 0 1
5 8 11 0 1 3 3
54 57 67 18 25 0 8
3 7 11 1 3 0 5
5 5 7 0 1 2 0
7 0 8 0 1 0 2
5 7 11 5 7 0 2
3 3 3 2 2 0 3
5 8 12 0 1 3 1
16 16 16 10 10 0 5
5 8 11 0 1 3 3
5 0 5 0 1 0 2
4 7 10 0 1 3 4
10 10 10 7 7 0 4
11 8 8 0 2 0 1
11 8 8 0 2 0 2
11 8 8 0 2 0 3
11 8 8 0 2 0 3
11 8 8 0 2 0 4
2 0 0 0 2 3 1
0 0 0 0 0 0 0</RA>
Plan 1
<TA>0,1,2,8,3,16,14,18,15,11,20,22,7,12,21,5,24,25,26,27,28,19,23,6,9,13,10,4,17,29,30</TA>
<RA>0 0 0 0 0 0 0
15 0 19 0 7 0 2
27 0 35 0 13 0 5
18 0 24 0 5 5 3
3 3 3 2 2 0 3
9 9 9 6 6 0 8
7 7 7 4 4 0 5
5 7 10 3 1 0 7
32 0 43 0 16 0 6
7 7 7 4 4 0 6
3 3 3 2 2 0 1
5 7 10 0 1 3 4
36 38 53 13 17 0 7
3 7 9 1 3 0 8
5 5 7 0 1 2 0
7 0 7 0 1 0 4
5 7 10 5 6 0 4
3 3 3 2 2 0 8
5 7 10 0 1 3 1
14 14 1

In [14]:
#Проверка части на генерированные данные
B, land, H = population[0][2], population[0][3], population[0][4] 
for it in llm_model.get_history():
    role, t = it.values()
    print(role)
    if role != 'llm':
        continue
    print(t[:50])
    TA, RA = parse_chromosome(t)
    offspring_chrom  = ChromosomeType((TA, RA, B, land, H))
    try:
        if check_chromosome(wg, contractors, offspring_chrom):
            print('OK')
    except:
        print('Ошибка')



system
llm
Plan 0
<TA>0,1,2,8,3,16,14,18,15,11,20,22,7,12,21,
OK
llm
Plan 2
<TA>0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,1
OK
llm
Plan 0
<TA>0,1,2,8,3,16,14,18,15,11,20,22,7,12,21,
OK
llm
Plan 0
<TA>0,1,2,8,3,16,14,18,15,11,20,22,7,12,21,
OK
llm
<TA>0,1,2,8,3,16,14,18,15,11,20,22,7,12,21,5,24,25
OK
llm
Plan 0
<TA>0,1,2,8,3,16,14,18,15,11,20,22,7,12,21,
OK


In [21]:
[p[2] for p in population_g]

[False, True, False, False, False, False, False]

In [23]:
f = TimeFitness()

for chrom_d in list(init_choromosome.values()):
    chrom, _, _ = chrom_d
    fitness = f.evaluate(chrom, evaluator=toolbox.evaluate_chromosome)
    print(fitness)
    

(41,)
(69,)
(69,)
(133,)
(96,)
(69,)
(69,)


In [24]:
[(fit, b_f) for (p, fit, b_f) in population_g]

[((133,), False),
 ((133,), True),
 ((96,), False),
 ((69,), False),
 ((69,), False),
 ((69,), False),
 ((69,), False)]

In [ ]:
4# population = []

# for chrom_d in list(init_choromosome.values()):
#     chrom, _, _ = chrom_d
#     population.append(chrom)

    